## Time series and forecasting models


### Time series as stochastic processes
* $X(t)$ or $X_t$, $t =1, 2, \dots$ (random variables)
* *Mean*. $\mu(t) = E [X(t)] $
* *Variance*. $\sigma^2(t) = {\rm Var}[X(t)]$
* *Autocovariance*. $\gamma(t_1, t_2) = E\{[X(t_1) - \mu(t_1)][X(t_2) - \mu(t_2)]\}$
* A time series is called (weakly) stationary if its mean is a constant and autocovariance function depends only on the lag $t_2-t_1$.
* It follows its variance is also a constant (corresponding to autocovariance for lag 0).  
* *Autocorrelation* at lag $\tau$: $\rho(\tau) = {\rm corr}(X_t, X_{t+\tau})$
* Sample mean, sample autocovariance, sample autorrelation: \\
Use sample average to replace the expectation in the definition

### White noise processes
* $Z_1, Z_2, \dots$, mutually independent and identically distributed
* mean 0, variance $\sigma_Z^2$
* autocorrelation at lag $k$:
$\rho(k) = 1$ if $k=0$, $\rho(k)=0$ otherwise
* sometimes weaker assumption is used: \\
$Z_1, Z_2, \dots$, mutually uncorrelated 
* it is the simplest stationary process
* it is used as building blocks for more complicated models

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import arma_generate_sample
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install statsmodels


**Example**: Generate a white noise process and draw a time series plot of the process. 

In [ ]:
# seed random number generator
np.random.seed(42)

# create white noise array
white_noise_array = np.random.normal(loc=0, scale=1, size= 200)

# draw a time series plot
plt.figure(figsize=(10,2))
plt.plot(white_noise_array)
plt.title('White Noise Process')
plt.show()


In [ ]:
mu, sigma = 0, 0.1 # mean and standard deviation
s = np.random.normal(mu, sigma, 1000)

In [ ]:
abs(sigma - np.std(s, ddof=1))

### Random walks
* $\{Z_t, t= 1, 2, \dots \}$ is a white noise process with mean $\mu$ and variance $\sigma_Z^2$
* $\{X_t, t=1, 2, \dots \}$ is a random walk if \\
$$X_t = X_{t-1} + Z_t$$
* the process is customarily started at zero when $t=0$, so that $X_1 = Z_1$
* $X_t = \sum_{i=1}^t Z_i$
* $E(X_t) = t\mu$, ${\rm Var}(X_t) = t\sigma_Z^2$, both changing with $t$
* a random walk is non-stationary
* the first difference of a random walk is a white noise, which is stationary \\
$$X_t - X_{t-1} = Z_t$$

**Example**: Generate a random walk process and draw a time series plot. Compare with the time series plot of the first difference of random walk.

Two methods for generating random walk process are presented. 
1. Use a for loop, according to the model equation of the white noise process.
2. Take the cumulative sum of the white noise process. (The looping operation is implicit.)

In [ ]:
# Method 1: Generate a random walk process according to the definition X_t = X_{t-1} + Z_t using a for loop
## seed random generator
np.random.seed(42)

## create random walk process
random_walk_array = np.zeros(200)
random_walk_array[0] = np.random.normal()
for i in range(1, 200):
  random_walk_array[i] = random_walk_array[i-1] + np.random.normal()


In [ ]:
# Method 2: An easier way to generate a random walk process by taking cumulative sum of the white noise
# We use white_noise_array enerated in the previous example. (We need to run the code chunk for generating the white noise before running this trunk.)
random_walk_array = np.cumsum(white_noise_array)

In [ ]:
# Visualization
plt.figure(figsize=(10,2))
plt.plot(random_walk_array)
plt.title('Random Walk Process')
plt.show()

### Moving Average processes
* $\{Z_t, t=1, 2, \dots\}$ is a white noise process
* $\{X_t, t=1, 2, \dots\}$ is a moving average process of order $q$, MA($q$), if
$$X_t = \beta_0 Z_t  + \beta_1 Z_{t-1} + \dots + \beta_q Z_{t-q}$$
where $\beta_i$ are constants.
* Usually $Z$s are scaled so that $\beta_0 =1$.
* $E(X_t) =0$
* ${\rm Var} (X_t) = \sigma_Z^2 \sum_{i=1}^q \beta_i^2$.
* Autocorrelation function (ACF)
$$ \rho(k) = {\rm corr} (X_t, X_{t+k}) = \begin{cases} 0  & k >q \\ 
 \sum_{i=1}^{q-k} \beta_i\beta_{i+1}/\sum_{i=1}^q \beta_i^2 & k =0, \dots, q\\
\rho(-k) & k<0 \end{cases} $$
* MA($q$) is a stationary process (because the above quantities do not depend on $t$).
* The autocorrelation "cuts off" at lag $q$. -- *This property can be used to identify the order of a MA process.*
* Sample autocorrelation will not "cuts off" exactly becaues of randomness

**Example**: Generate a MA($q$) process with $q = 2$ and draw a time series plot of the process. 

Two methods for generating a MA($2$) process are presented. 
1. The MA($2$) process is generated according to its defining equation
$$
X_t = \beta_0Z_t + \beta_1 Z_{t-1} +\beta_2 Z_{t-2} 
$$
where $\beta_0 = 1, \beta_1 = 0.8$ and $\beta_3 = 0.5$. 
2. The MA($2$) process is generated using `arma_generate_sample` function in the module `arma_process`. The definition of ARMA model and the relation between ARMA and MA model will be introduced later. 


In [ ]:
# Method 1: Generate a MA(2) process according to its model equation.
## seed random generator
np.random.seed(42)
##  create ma(2) array
order_ma = 2 # order
ma_coef = [1.0, 0.8, 0.5] # MA coefficient [beta_0, beta_1, beta_2]
## create random noise array
white_noise = np.random.normal(size=1000)
## initialization
ma_array = np.zeros(1000)
ma_array[0:2] = [0,0]
## assignment
for i in range(2, 1000):
    ma_array[i] = ma_coef[0] * white_noise[i] + ma_coef[1] * white_noise[i-1] + ma_coef[2] * white_noise[i-2]

In [ ]:
# Method 2: Generate a MA(2) process using arma_generate_sample function

from statsmodels.tsa.arima_process import arma_generate_sample

np.random.seed(42)
ma_coef = [1.0,0.8,0.5]
ma_array = arma_generate_sample(ar = [1.0],ma = ma_coef,nsample =  1000)

In [ ]:
#  Visualization
ma_array_300 = ma_array[700:1000]
plt.figure(figsize=(10,2))
plt.plot(ma_array_300)
plt.title('Moving Average Process')
plt.show()
## Check the ACF plot
plot_acf(pd.DataFrame(ma_array_300))
plt.show()

The first two autocorrelation coefficients are significantly different from zero, suggesting an MA(2) process.

### Autoregressive processes
* $\{Z_t, t=1, 2, \dots\}$ is a white noise process
* $\{X_t, t=1, 2, \dots\}$ is an autoregressive process of order $p$, AR($p$), if
$$X_t = \alpha_1 X_{t-1} + \dots \alpha_p X_{t-p} + Z_{t}$$
where $\alpha_i$ are constants.
* This is link a multiple regression model, but $X_t$ is regressed on past values of $X_t$. -- This explains the prefix "auto".
* $E(X_t)=0$
* It is stationary under some conditions on the coefficients $\alpha_i$s.


**Example**: Generate a AR(1) process with multiple choices of $\alpha$ and draw a time series plot of the process. 

Two methods to generate AR($1$) process are presented. 
1. The AR($1$) process is generated according to the definition such that
$$
X_t = Z_t + \alpha X_{t-1} 
$$
2. The AR($1$) process is generated using `arma_generate_sample` function in the module `arma_process`.

In [ ]:
# Method 1: Generate the AR(1) process according to its definition
def Autoregressive(alpha, seed=42):
   # seed random generator
    np.random.seed(seed)
   # create AR(1) array
    ar = np.zeros(100)
    ar[0] = np.random.normal(loc=0, scale=1.0/(1-alpha**2), size=1)
    ## assignment
    for i in range(1, 100):
        ar[i] = alpha * ar[i-1] + np.random.normal(size=1)
    return ar

auto_reg1 = Autoregressive(0.8)
auto_reg2 = Autoregressive(0.3)
auto_reg3 = Autoregressive(-0.8)

In [ ]:
# Method 2: Generate the AR(1) using the arma_generate_sample function

from statsmodels.tsa.arima_process import arma_generate_sample

ar_coef = [1.0, -0.8] # Note we need to add a negative sign in front of the AR coefficient.
ma_coef = [1.0]
auto_reg = arma_generate_sample(ar = ar_coef, ma = ma_coef, nsample =  100)

# If we need to repeat the process for different values of the AR coefficient, it is better to write a python function.

def Autoregressive(alpha, seed = 42):
  np.random.seed(seed)
  ar_coef = [1.0, -1.0*alpha]
  ma_coef = [1.0]
  ar_array = arma_generate_sample(ar = ar_coef,ma = ma_coef,nsample =  100)
  return ar_array

auto_reg1 = Autoregressive(0.8)
auto_reg2 = Autoregressive(0.3)
auto_reg3 = Autoregressive(-0.8)

In [ ]:
# Visualization of the time series

plt.figure(figsize=(18,2))
plt.subplot(131)
plt.plot(auto_reg1)
plt.title('Auto_reg coefficient=0.8')
plt.subplot(132)
plt.plot(auto_reg2)
plt.title('Auto_reg coefficient=0.3')
plt.subplot(133)
plt.plot(auto_reg3)
plt.title('Auto_reg coefficient=-0.8')
plt.show()

In [ ]:
# Plots of sample autocorrelation function. We need to use the Python module matplotlib.gridspect to arrange multiplots.
import matplotlib.gridspec as gridspec

fig = plt.figure(tight_layout = True, figsize = (12, 4))
gs = gridspec.GridSpec(1, 3)
ax = fig.add_subplot(gs[0,0])
plot_acf(pd.DataFrame(auto_reg1), ax=ax, title='Autocorrelation case: alpha=0.8')
ax = fig.add_subplot(gs[0,1])
plot_acf(pd.DataFrame(auto_reg2), ax=ax, title='Autocorrelation case: alpha=0.3')
ax = fig.add_subplot(gs[0,2])
plot_acf(pd.DataFrame(auto_reg3), ax=ax, title='Autocorrelation case: alpha=-0.8')
plt.show()


### Some theory of AR(1) processes
* $X_t = \alpha X_{t-1} + Z_t$
* Backward shift operator $B$: $B X_t = X_{t-1}$.
* AR(1) model can be rewritten as $(1-\alpha B) X_t = Z_t$
* MA($\infty$) representation
$$ \begin{eqnarray*} X_t &=& Z_t/ (1-\alpha B) \\ 
&=& (1 + \alpha B + \alpha^2 B^2 + \dots) Z_t\\
&=& Z_t + \alpha Z_{t-1} + \alpha^2 Z_{t-2} + \dots \end{eqnarray*} $$
* This representation can be used to derive the autocorrelation function
$$\rho(k) = {\rm corr}(X_t, X_{t+k}) = \alpha^k, \quad k = 0, 1, 2, \dots $$
* Need $|\alpha| < 1 $ for the process to be stationary.
* If $\alpha$ is close to $0$, the autocorrelaton decays quickly to 0.
* If $\alpha$ is negative, the autocorrelaton alternates between positive and negative values.

### Indentify the order of an AR process by inspecting the Partial Autocorrelation (PACF) plot.

Partial autocorrelation can be imagined as the correlation between the series and its lag, after excluding the contributions from the intermediate lags. So, PACF sort of conveys the pure correlation between a lag and the series. That way, you will know if that lag is needed in the AR term or not.

So what is the formula for PACF mathematically?

Partial autocorrelation of lag k of the series $\{X_t\} $ is the coefficient $\alpha_k$ in the following autoregression equation:

$$X_t = \alpha_0 + \alpha_1 X_{t-1} + \alpha_2 X_{t-2} + \dots \alpha_k X_{t-k} + Z_t$$

How to find the number of AR terms?

Any autocorrelation in a stationarized series can be rectified by adding enough AR terms. So, we initially take the order of AR term to be equal to as many lags that crosses the significance limit in the PACF plot.




### Mixed ARMA models
* A mixed autoregressive/moving-average process containing $p$ AR terms and $q$ MA terms is called an ARMA process of order $(p,q)$.
* Its equation
$$ X_t = \alpha_1 X_{t-1} + \dots + \alpha_p X_{t-p} + Z_t + \beta_1 Z_{t-1} + \dots + \beta_q Z_{t-q}$$
* Define two polynomials 
$$
\begin{eqnarray*} 
\phi(B) & = & 1 -\alpha_1 B - \dots - \alpha_p B^p\\
\theta(B) & = & 1 + \beta_1 B + \dots + \beta_q B^q
\end{eqnarray*}
$$
* If $B$ is the backward shift operator, ARMA model can be written as
$$ \phi(B) X_t = \theta(B) Z_t
$$
* The process is stationary under certain conditions on the coefficients.
* Condition: The (complex) roots of $\phi(B)$ =0 and $\theta(B) = 0$ lie outside the unit circle.


**Example**: Generate a sample path for the following ARMA(2,1) process and draw a time series plot.
$$
X_t = \frac{5}{6}X_{t-1}-\frac{1}{6}X_{t-2}+Z_t + \frac{1}{2}Z_{t-1}
$$
The equation can also be written as
$$
X_t - \frac{5}{6}X_{t-1} + \frac{1}{6}X_{t-2} = Z_t + \frac{1}{2}Z_{t-1}
$$


In [ ]:
from statsmodels.tsa.arima_process import arma_generate_sample

# seed random generator
np.random.seed(192)

# Generate arma(2,1) array using Python arma_generate_sample function 
ar_coef = [1, -5/6, 1/6] # The coefficient for autoregressive lag polynomial phi(B), including zero lag
ma_coef = [1, 1/2] # The coefficient for moving-average lag polynomial theta(B), including zero lag
my_arma_array = arma_generate_sample(ar_coef, ma_coef, 250) #sample size = 250

In [ ]:
# Visualization
plt.figure(figsize=(12,2))
plt.plot(my_arma_array)
plt.title('ARMA(2,1) process')
plt.show()

In [ ]:
## ACF and PACF plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.gridspec as gridspec

fig = plt.figure(tight_layout = True, figsize = (12, 4))
gs = gridspec.GridSpec(1, 2)
ax = fig.add_subplot(gs[0,0])
plot_acf(my_arma_array,title='ACF of ARMA(2,1)', ax = ax)
ax = fig.add_subplot(gs[0,1])
plot_pacf(my_arma_array,title='PACF of ARMA(2,1)',ax = ax)
plt.show()

We observe there are some significant ACF coefficients and PACF coefficients.

### Why ARMA models?
* A stationary time series may often be adequately modelling by an ARMA model involving fewer parameters than a pure MA or AR process by itself.
* This is calle the **Principle of Parsimony.**
* We want to find a model with as few parameters as possible, but which gives an adequate representation of the data at hand.

### Integrated ARMA (ARIMA) model
* In practice, many time series are non-stationary.
* Differencing the series can remove the non-stationary sources of variation.
* First difference: $\Delta X_t = X_t - X_{t-1} = (1-B) X_t$
* Second difference: $\Delta^2 X_t = \Delta X_t - \Delta X_{t-1} = (1-B^2) X_t$
* $d$-th difference: $W_t = \Delta^d X_t = (1-B)^d X_t$ 
* $W_t$ is ARMA$(p,q)$:
$$ W_t = \alpha_1 W_{t-1} + \dots + \alpha_p W_{t-p} + Z_t + \beta_1 Z_{t-1} + \dots + \beta_q Z_{t-1}$$
* $X_t$ is called an ARIMA process of order $(p,d,q)$ -- autoregressive integrated moving average
* $X_t$ is not stationary. Its $d$-th difference $W_t$ is stationary.
* First differencing is often found to be adequate in pratice.
* The random walk is ARIMA (0,1,0). --It is white noise after taking first diference.

**Example**: Generate an ARIMA(2,1,1) process based on `my_arma_array` (ARMA(2,1)) created in the previous example and draw the time series plot of the process. 

In [ ]:
# Generate a sample path for an ARIMA(2,1,1) process using the sample path of an ARMA(2,1) process
# Use cumsum function in numpy
my_arima_array = np.cumsum(my_arma_array) 

# visualization
plt.figure(figsize=(12,4))
plt.subplot(121)
plt.plot(my_arma_array)
plt.title('ARMA(2,1) process')
plt.subplot(122)
plt.plot(my_arima_array)
plt.title('ARIMA(2,1,1) process')
plt.show()

We observe that ARMA(2,1) is stationary; its path varies around its mean. ARIMA(2,1,1) is not stationary; its path shows a trending behavior (called stochastic trend). We can use the KPSS and ADF tests to check the stationarity of a time series.

### ADF test

The augmented Dickey–Fuller (ADF) statistic, used in the test, is a negative number. The more negative it is, the stronger the rejection of the hypothesis that there is a unit root at some level of confidence

ADF test is used to determine the presence of unit root in the series, and hence helps in understand if the series is stationary or not. The null and alternate hypothesis of this test are:

 * Null Hypothesis: The series has a unit root, or the series is non-stationary.

* Alternate Hypothesis: The series has no unit root, or the series is stationary.

If the null hypothesis in failed to be rejected, this test may provide evidence that the series is non-stationary.

In [ ]:
# ADF test for checking the stationarity of a time series.
from statsmodels.tsa.stattools import adfuller

def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
adf_test(my_arima_array)

p-value is 0.52, which much more than 5% confidence level. The null hypothesis can not be rejected. Hence the series is non-stationary.

### KPSS test

In econometrics, Kwiatkowski–Phillips–Schmidt–Shin (KPSS) tests are used for testing a null hypothesis that an observable time series is stationary around a deterministic trend (i.e. trend-stationary) against the alternative of a unit root.


KPSS is another test for checking the stationarity of a time series. The null and alternate hypothesis for the KPSS test are different from that of the ADF test.
 * Null Hypothesis: The process is trend stationary.
 * Alternate Hypothesis: The series has a unit root (series is not stationary).

In [ ]:
# KPSS test for checking the stationarity of a time series.
from statsmodels.tsa.stattools import kpss

def kpss_test(timeseries):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression='c')
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    print (kpss_output)

In [ ]:
kpss_test(my_arima_array)

p-value is 0.01, which less than 5% confidence level. The null hypothesis can be rejected. Hence the series is non-stationary.

In [ ]:
## ACF and PACF plots

import matplotlib.gridspec as gridspec

fig = plt.figure(tight_layout = True, figsize = (14, 8))
gs = gridspec.GridSpec(2, 2)
ax = fig.add_subplot(gs[0,0])
plot_acf(my_arma_array,title='ACF of ARMA(2,1)', ax = ax)
ax = fig.add_subplot(gs[0,1])
plot_pacf(pd.DataFrame(my_arma_array),title='PACF of ARMA(2,1)',ax = ax)
ax = fig.add_subplot(gs[1,0])
plot_acf(my_arima_array,title='ACF of ARIMA(2,1,1)',ax = ax)
ax = fig.add_subplot(gs[1,1])
plot_pacf(my_arima_array,title='PACF of ARIMA(2,1,1)',ax = ax)
plt.show()

The ACF of the ARIMA(2,1,1) process decays very slowly, suggesting it is an integrated process (non-stationary). 

### The meaning of *p*, *d* and *q* in an ARIMA model 

*p* is the order of the Auto Regressive (AR) term. It refers to the number of lags of Y to be used as predictors.

### The meaning of *d*

- The term **Auto Regressive** in ARIMA means it is a linear regression model that uses its own lags as predictors. Linear regression models, as we know, work best when the predictors are not correlated and are independent of each other. So we need to make the time series stationary.

- The most common approach to make the series stationary is to difference it. That is, subtract the previous value from the current value. Sometimes, depending on the complexity of the series, more than one differencing may be needed.

- The value of d, therefore, is the minimum number of differencing needed to make the series stationary. If the time series is already stationary, then d = 0.

#### The meaning of *q*

**q** is the order of the Moving Average (MA) term. It refers to the number of lagged forecast errors that should go into the ARIMA Model.

#### Finding the order of differencing

-  An over differenced series may still be stationary, which in turn will affect the model parameters.So we should determine the right order of differencing. 
-  The right order of differencing is the minimum differencing required to get a near-stationary series which roams around a defined mean and the ACF plot reaches to zero fairly quick.
- If the autocorrelations are positive for many number of lags (10 or more), then the series needs further differencing. On the other hand, if the lag 1 autocorrelation itself is too negative, then the series is probably over-differenced.
- If we can’t really decide between two orders of differencing, then we go with the order that gives the least standard deviation in the differenced series.


#### Finding the order of the AR term (p)

- We will find out the required number of AR terms by inspecting the Partial Autocorrelation (PACF) plot.
- Partial autocorrelation can be imagined as the correlation between the series and its lag, after excluding the contributions from the intermediate lags. So, PACF sort of conveys the pure correlation between a lag and the series. This way, we will know if that lag is needed in the AR term or not.



#### Finding the order of the MA term (q)

- We should use the ACF plot for the number of MA terms. An MA term is technically, the error of the lagged forecast.
- The ACF tells how many MA terms are required to remove any autocorrelation in the stationarized series.


### Fitting the ARIMA(p,d,q) model

Once you have determined the orders $p,d,q$ in ARIMA model, you can fit the ARIMA model. 

**Example**: Fit a ARIMA model to the `arima_array`(ARIMA(2,1,1)) data that we created in the previous example using `ARIMA()` function. 

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Split sample. Use the first 200 data points in my_arima_array as training data and the last 50 as test data. 
my_arima_array_train = my_arima_array[:200] 
my_arima_array_test = my_arima_array[200:]

# fit the ARIMA model
model = ARIMA(my_arima_array_train,order=(4, 1, 3))
#model_fit = model.fit(disp=0)

# print the fitting result
print(model.fit().summary())

### Diagnostics
After fitting the ARIMA model to data, we can do the following checks on the residuals to see if the fitted model is adequate. 

1. Plot the residuals to ensure there are no patterns (that is, look for constant mean and variance).

2. Series correlation. Plot acf and pacf to ensure there is no significant coefficient. 

**Example**: Diagnose the model fitting result. The residuals can be obtained by .resid function. 

In [ ]:
residuals = pd.DataFrame(model.fit().resid)

# Generate the residual plot
fig = plt.figure(tight_layout = True, figsize = (10, 4))
gs = gridspec.GridSpec(1, 2)
ax = fig.add_subplot(gs[0,0])
residuals.plot(title="Residuals",ax=ax, legend=False)
ax = fig.add_subplot(gs[0,1])
residuals.plot(kind='kde',title='Density',ax=ax,legend = False)
plt.show()

We observe there is no observious pattern in the residuals plot and the center of the density of residuals lies around zero.

In [ ]:
# Generate acf and pacf plot
fig = plt.figure(tight_layout = True, figsize = (10, 4))
gs = gridspec.GridSpec(1, 2)
ax = fig.add_subplot(gs[0,0])
plot_acf(residuals,ax = ax)
ax = fig.add_subplot(gs[0,1])
plot_pacf(residuals,ax = ax)
plt.show()

The autocorrelation and partial autocorrelation plot suggest that there is no significant coefficient. Therefore, the model fitted to the data is adequate. 

### ARIMA forecasting
With the determined orders $p,d,q$ and the fitting ARIMA model, we can then forecast the future.

**Example**: Forecast the future trend based on the `model_fit` we derived in the previous example and check whether the forecasting is valid by comparing the forecasts and the `arima_array_test`. The function `.forecast` will be used and it will return three arrays: 

1. forecast array: array of out of sample forecasts. 
2. stderr array: array of the standard errorof the forecasts.  
3. conf_int array: 2d array of the confidence interval for the forecasts.

In [ ]:
n_pred = 50 # Lead time, or forecasting horizon -- the number of steps ahead for out of sample forecast 
pred_array, se_array, CI_array = model.fit().forecast(steps=n_pred,alpha=0.05) # alpha: confidence level



In [ ]:

pred_array_index = range(200,250)

plt.figure(figsize=(12,4))
plt.plot(my_arima_array)
plt.plot(pred_array_index, pred_array, color = "red")
plt.fill_between(pred_array_index, CI_array[:,0], CI_array[:,1], color = "k", alpha = .15 )
plt.title('Forecast of the simulated data')
plt.show()

### The Box-Jenkins approach for time series forecasting
* (1) *Model identification* \\
Examine the data to see which member of the class of ARIMA processes appears to be most appropriate. Difference the data. Check the autocorrelation and partical autocorrelation. 
* (2) *Estimation* \\
Estimate the parameters of the chosen model.
* (3) *Diagnostic checking* \\
Examine the residuals from the fitted model to see if there is evidence of non-randomness. 
* (4) *Consideration of alternative models if necessary*

### Automatic determination of the orders of an ARIMA model
The manually iterative fitting procedure is an art and requires experience. The function below searches over multiple combinations of $p,d,q$ parameters and choose the optimal ARIMA model that has the least value of the AIC criterion.

In [ ]:
# Find the optimal ARIMA model that has the least value of the AIC criterion.
# Return: (p, d, q) that minimizes AIC criterion.

import itertools
def arima_order(data, order=10, verbose=True):
  p=d=q=range(0,order)
  pdq = list(itertools.product(p,d,q))
  warnings.filterwarnings("ignore")
  aic_pair={}
  min_aic=9999999
  min_order=(0,0,0)
  for o in pdq:
    try:
      model_arima = ARIMA(data, order=o)
      model_arima_fit=model_arima.fit()
      fit_aic = model_arima_fit.aic
      if verbose: print(o, "AIC=", fit_aic)
      if not np.isnan(fit_aic):
        if min_aic>fit_aic:
          min_aic = fit_aic
          min_order=o
      aic_pair.update({o, rmodel_arima_fit.aic})
    except:
      continue
  return min_order

In [ ]:
arima_order(my_arima_array_train, order=5)